In [449]:
import pandas as pd
import numpy as np
import json
import re
import ast

dfmovies = pd.read_csv(r'..\movies_dataset.csv')
dfmovies.head(5)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3016\3743336966.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfmovies = pd.read_csv(r'..\movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [450]:
# eliminamos los campos que no se consideran importantes o que son redundantes para el analisis
columns_to_drop = ["original_title", "homepage", "adult", "imdb_id", "video", "poster_path", 'tagline', 'runtime', 'status', 'overview']
dfmovies = dfmovies.drop(columns=columns_to_drop, errors='ignore')

# Seleccionar las columnas relevantes
columnas_seleccionadas = ['genres', 'belongs_to_collection', 'release_date', 'original_language', 
                          'popularity', 'production_companies', 'production_countries', 
                          'spoken_languages', 'id', 'vote_average', 'vote_count']

# Crear un nuevo DataFrame con las columnas seleccionadas
dfmovies_reducido = dfmovies[columnas_seleccionadas].copy()

In [451]:
# Reemplazar nulos por 0 y convertir las columnas 'revenue' y 'budget'.
dfmovies_reducido = dfmovies.copy()  # Trabajar sobre una copia para evitar el SettingWithCopyWarning

# Filtrar las filas que tienen valores no numéricos
dfmovies_reducido = dfmovies_reducido[pd.to_numeric(dfmovies_reducido['revenue'], errors='coerce').notnull()]
dfmovies_reducido = dfmovies_reducido[pd.to_numeric(dfmovies_reducido['budget'], errors='coerce').notnull()]

# Convertir las columnas 'revenue' y 'budget' a enteros después de filtrar
dfmovies_reducido['revenue'] = dfmovies_reducido['revenue'].astype(float).fillna(0).astype(int)
dfmovies_reducido['budget'] = dfmovies_reducido['budget'].astype(float).fillna(0).astype(int)
dfmovies_reducido['revenue']
dfmovies_reducido['budget']

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45460, dtype: int32

In [452]:
# Función para extraer el nombre de la colección
def extraer_nombre_manualmente(row):
    if pd.isna(row) or isinstance(row, float):
        return 'Dato no disponible'
    
    if isinstance(row, str):
        try:
            # Intentar extraer el nombre usando comillas simples o dobles
            match = re.search(r"'name':\s*'([^']+)'|\"name\":\s*\"([^\"]+)\"", row)
            if match:
                # Devolver el grupo correspondiente que coincida
                return match.group(1) if match.group(1) else match.group(2)
            else:
                return 'Dato no válido'
        except Exception:
            return 'Dato no válido'
    
    return 'Dato no disponible'

# Aplicar la función a cada fila en el DataFrame
dfmovies_reducido['nombre_coleccion'] = dfmovies_reducido['belongs_to_collection'].apply(extraer_nombre_manualmente)

# Mostrar el resultado
dfmovies_reducido[['nombre_coleccion']].head(5)

,nombre_coleccion
0,Toy Story Collection
1,Dato no disponible
2,Grumpy Old Men Collection
3,Dato no disponible
4,Father of the Bride Collection


In [453]:
dfmovies_reducido['revenue'] = dfmovies_reducido['revenue'].apply(lambda x: abs(x)) # pasamos los valores negativos a positivos
dfmovies_reducido['revenue']

0        373554033
1        262797249
2                0
3         81452156
4         76578911
           ...    
45461            0
45462            0
45463            0
45464            0
45465            0
Name: revenue, Length: 45460, dtype: int64

In [454]:
# Función para extraer los nombres de los géneros
def extraer_generos(row):
    if pd.isna(row) or row.strip() == '[]':
        return 'Sin género'
    
    try:
        # Buscar todos los nombres de géneros en la cadena usando una expresión regular
        generos = re.findall(r"'name':\s*'([^']+)'", row)
        return ', '.join(generos) if generos else 'Sin género'
    except Exception:
        return 'Dato no válido'

# Aplicar la función a la columna 'genres' y guardar el resultado en la columna 'generos'
dfmovies_reducido['generos'] = dfmovies_reducido['genres'].apply(extraer_generos)

# Verificar que los géneros han sido correctamente extraídos
dfmovies_reducido[['genres', 'generos']].head(5)

,genres,generos
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Animation, Comedy, Family"
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","Adventure, Fantasy, Family"
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","Romance, Comedy"
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Comedy, Drama, Romance"
4,"[{'id': 35, 'name': 'Comedy'}]",Comedy


In [455]:
# Función para extraer los nombres de los países de producción
def extraer_paises(row):
    if pd.isna(row) or row.strip() == '[]':
        return 'Sin país de producción'
    
    try:
        # Buscar todos los nombres de países en la cadena usando una expresión regular
        paises = re.findall(r"'name':\s*'([^']+)'", row)
        return ', '.join(paises) if paises else 'Sin país de producción'
    except Exception:
        return 'Dato no válido'

# Aplicar la función a la columna 'production_countries'
dfmovies_reducido['production_countries_clean'] = dfmovies_reducido['production_countries'].apply(extraer_paises)

In [456]:
# Función para extraer los nombres de las empresas de producción
def extraer_empresas(row):
    if pd.isna(row) or row.strip() == '[]':
        return 'Sin empresa de producción'
    
    try:
        # Buscar todos los nombres de empresas en la cadena usando una expresión regular
        empresas = re.findall(r"'name':\s*'([^']+)'", row)
        return ', '.join(empresas) if empresas else 'Sin empresa de producción'
    except Exception:
        return 'Dato no válido'

# Aplicar la función a la columna 'production_companies'
dfmovies_reducido['production_companies_clean'] = dfmovies_reducido['production_companies'].apply(extraer_empresas)

In [457]:
# Función para extraer los nombres de los idiomas hablados
def extraer_idiomas(row):
    if pd.isna(row) or row.strip() == '[]':
        return 'Sin idioma'
    
    try:
        # Convertir la cadena en una lista de diccionarios
        lista_idiomas = ast.literal_eval(row)
        # Extraer los valores de 'name' de cada diccionario en la lista
        idiomas = [idioma['name'] for idioma in lista_idiomas if 'name' in idioma]
        return ', '.join(idiomas) if idiomas else 'Sin idioma'
    except (ValueError, SyntaxError):
        return 'Dato no válido'

# Aplicar la función a la columna 'spoken_languages'
dfmovies_reducido['spoken_languages_clean'] = dfmovies_reducido['spoken_languages'].apply(extraer_idiomas)

In [458]:
# Eliminar las columnas originales si ya no son necesarias porque ya pasamos los datos a otras columnas y campos como status ya serian redudantees por relase_date y release_year
dfmovies_reducido.drop(columns=['production_countries', 'production_companies', 'genres', 'spoken_languages', 'belongs_to_collection'], inplace=True)

In [459]:
#verificar que mis campos numericos sean numericos
dfmovies_reducido[['budget', 'revenue', 'popularity', 'vote_average', 'vote_count']] = dfmovies_reducido[['budget', 'revenue', 'popularity', 'vote_average', 'vote_count']].apply(pd.to_numeric, errors='coerce')

In [460]:
dfmovies_reducido.dropna(subset=['release_date'], inplace=True) # eliminamos nulos de la columna release_date

# Convertir columnas a los tipos de datos deseados
dfmovies_reducido['release_date'] = pd.to_datetime(dfmovies_reducido['release_date'], errors='coerce')
dfmovies_reducido['budget'] = pd.to_numeric(dfmovies_reducido['budget'], errors='coerce').fillna(0).astype(int)
dfmovies_reducido['revenue'] = pd.to_numeric(dfmovies_reducido['revenue'], errors='coerce').fillna(0).astype(int)
dfmovies_reducido['popularity'] = pd.to_numeric(dfmovies_reducido['popularity'], errors='coerce').fillna(0.0).astype(float)

# Reemplazar fechas inválidas en 'release_date' con un valor por defecto
dfmovies_reducido['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)

# Extraer el año de 'release_date' y convertir a entero
dfmovies_reducido['release_year'] = dfmovies_reducido['release_date'].dt.year.astype(int)

# Verificación rápida del resultado
dfmovies_reducido[['release_date', 'release_year']].head(5)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3016\3326991025.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfmovies_reducido['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)


,release_date,release_year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995
3,1995-12-22,1995
4,1995-02-10,1995


In [461]:
# Identificar el idioma más común
idioma_mas_comun = dfmovies_reducido['original_language'].mode()[0]

# Llenar los valores nulos con el idioma más común
dfmovies_reducido['original_language'].fillna(idioma_mas_comun, inplace=True)

print(dfmovies_reducido.isnull().sum())

budget                        0
id                            0
original_language             0
popularity                    0
release_date                  0
revenue                       0
title                         0
vote_average                  0
vote_count                    0
nombre_coleccion              0
generos                       0
production_countries_clean    0
production_companies_clean    0
spoken_languages_clean        0
release_year                  0
dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_3016\2647718338.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfmovies_reducido['original_language'].fillna(idioma_mas_comun, inplace=True)


In [462]:
dfmovies_reducido.rename(columns={'id': 'movie_id'}, inplace=True)

In [463]:
# Guardar como archivo Parquet
dfmovies_reducido.to_parquet(r'..\Datasets\datasetmovies_reducido.parquet', index=False)

In [464]:
dfcredits = pd.read_csv(r'..\credits.csv')

# Crear una copia del DataFrame original
dfcredits_reducido = dfcredits.copy()

In [465]:
def extraer_nombres_actores(cast):
    # Verifica si 'cast' es una cadena de texto
    if isinstance(cast, str):
        try:
            # Convertir la cadena de texto a lista de diccionarios
            cast = ast.literal_eval(cast)
        except (ValueError, SyntaxError):
            return []
    
    # Verifica si 'cast' es una lista de diccionarios
    if isinstance(cast, list):
        return [actor.get('name', 'Sin nombre') for actor in cast if isinstance(actor, dict) and 'name' in actor]
    
    return []

# Aplicar la función a la columna 'cast'
dfcredits_reducido['actor_name'] = dfcredits_reducido['cast'].apply(extraer_nombres_actores)

# Explode para separar cada actor en una fila
dfcredits_reducido = dfcredits_reducido.explode('actor_name').reset_index(drop=True)

# Filtrar donde 'actor_name' no sea NaN (sin valores nulos)
dfcredits_reducido = dfcredits_reducido[dfcredits_reducido['actor_name'].notna()]

# Mostrar el resultado
dfcredits_reducido.head(5)

,cast,crew,id,actor_name
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Tom Hanks
1,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Tim Allen
2,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Don Rickles
3,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Jim Varney
4,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Wallace Shawn


In [466]:
# Convertir la columna 'crew' a una lista de diccionarios si es necesario
def procesar_crew(crew):
    if isinstance(crew, str):
        try:
            return json.loads(crew.replace("'", '"'))  # Convertir la cadena a JSON
        except json.JSONDecodeError:
            return []  # Si hay un error, devolver una lista vacía
    return crew if isinstance(crew, list) else []

# Aplicar la función a la columna 'crew' para procesar el formato
dfcredits_reducido['crew'] = dfcredits_reducido['crew'].apply(procesar_crew)

# Filtrar filas donde 'crew' es una lista vacía o es nulo
dfcredits_reducido = dfcredits_reducido[dfcredits_reducido['crew'].apply(len) > 0]

# Función para extraer el nombre y ID del director
def extraer_director(crew_list):
    director = next((miembro for miembro in crew_list if miembro.get('job') == 'Director'), None)
    if director:
        return director.get('name', 'Sin director'), director.get('id', None)
    return None, None  # Devolver None si no hay director

# Aplicar la función a la columna 'crew' y crear nuevas columnas 'director_name' y 'director_id'
dfcredits_reducido[['director_name', 'director_id']] = dfcredits_reducido['crew'].apply(extraer_director).apply(pd.Series)

# Filtrar filas donde 'director_name' o 'director_id' es None
dfcredits_reducido = dfcredits_reducido[dfcredits_reducido['director_name'].notna()]

# Eliminar filas duplicadas basadas en el 'director_name'
dfcredits_reducido = dfcredits_reducido.drop_duplicates(subset=['director_name']).reset_index(drop=True)

# Mostrar todos los nombres únicos de directores
dfcredits_reducido['director_name'].head(5)

0        Oliver Parker
1    Richard Loncraine
2       Claude Lelouch
3         Jafar Panahi
4         Henry Jaglom
Name: director_name, dtype: object

In [467]:
# Renombrar las columnas 'id' y 'director_id' a 'movie_id' y 'director_movie_id' respectivamente
dfcredits_reducido.rename(columns={'id': 'movie_id', 'director_id': 'director_movie_id'}, inplace=True)

# Eliminar las columnas 'cast' y 'crew' ya que hemos extraído la información relevante
dfcredits_reducido.drop(columns=['cast', 'crew'], inplace=True)

In [468]:
dfcredits_reducido['director_movie_id'].isnull().sum()

0

In [469]:
# Guardar como archivo Parquet
dfcredits_reducido.to_parquet(r'..\Datasets\datasetcredits_reducido.parquet', index=False)

In [470]:
# Cargar los archivos Parquet
df_credits = pd.read_parquet(r'..\Datasets\datasetcredits_reducido.parquet')
df_movies = pd.read_parquet(r'..\Datasets\datasetmovies_reducido.parquet')

In [471]:
# Renombrar la columna 'id' en df_movies a 'movie_id' para luego poder unir las dos a un dataset común mediante este campo
df_movies = df_movies.rename(columns={'id': 'movie_id'})

In [472]:
# Revisar cuántos valores NaN quedaron después de la conversión de movie_id
print(df_credits['movie_id'].isna().sum())
print(df_movies['movie_id'].isna().sum())

0
0


In [473]:
# Convertir 'movie_id' a un formato numérico para unificación
df_credits['movie_id'] = pd.to_numeric(df_credits['movie_id'], errors='coerce').astype('Int64')
df_movies['movie_id'] = pd.to_numeric(df_movies['movie_id'], errors='coerce').astype('Int64')

#El DataFrame final (df_unido) contiene solo las películas que estaban presentes tanto en df_credits como en df_movies, con todas sus columnas combinadas.
df_unido1 = pd.merge(df_credits, df_movies, on='movie_id', how='outer', indicator=True)
df_unido = df_unido1[df_unido1['_merge'] == 'both']

# Filtrar las películas que están en ambos DataFrames
df_unido = df_unido[df_unido['_merge'] == 'both']

# Eliminar la columna _merge si ya no es necesaria
df_unido = df_unido.drop(columns=['_merge'])

# Verificar el resultado final
print("Dimensiones del DataFrame final con películas en ambos DataFrames:", df_unido.shape)


# Filtrar solo las películas en inglés
df_unido = df_unido[df_unido['original_language'] == 'en']

Dimensiones del DataFrame final con películas en ambos DataFrames: (2378, 18)


In [474]:
df_unido.head(5)

,movie_id,actor_name,director_name,director_movie_id,budget,original_language,popularity,release_date,revenue,title,vote_average,vote_count,nombre_coleccion,generos,production_countries_clean,production_companies_clean,spoken_languages_clean,release_year
269,333,Rahul Khanna,Deepa Mehta,4760.0,0.0,en,2.015731,2002-01-01,0.0,Bollywood/Hollywood,6.1,4.0,Dato no disponible,"Comedy, Drama, Music, Romance",Canada,Different Tree Same Wood Productions,English,2002.0
891,1392,Zana Briski,Zana Briski,53384.0,350000.0,en,2.962370,2004-12-08,3515061.0,Born into Brothels,6.7,61.0,Dato no disponible,Documentary,United States of America,"HBO/Cinemax Documentary, thinkfilm, Creative V...","বাংলা, English",2004.0
936,1485,Michael Caine,Mike Hodges,17281.0,1814462.0,en,7.505268,1971-03-03,0.0,Get Carter,6.9,122.0,Dato no disponible,"Crime, Thriller",United Kingdom,Metro-Goldwyn-Mayer (MGM),English,1971.0
993,1587,Johnny Depp,Lasse Hallström,5306.0,11000000.0,en,10.391900,1993-12-17,10032765.0,What's Eating Gilbert Grape,7.5,943.0,Dato no disponible,"Romance, Drama",United States of America,Paramount Pictures,English,1993.0
1015,1618,Stephen Rea,Brian Gibson,18069.0,0.0,en,11.089962,1998-10-30,0.0,Still Crazy,7.1,35.0,Dato no disponible,"Comedy, Music",United Kingdom,"Columbia Pictures Corporation, The Greenlight ...","English, Magyar",1998.0


In [475]:
# Limpiar las columnas de texto
df_unido['actor_name'] = df_unido['actor_name'].str.strip().replace(r'\s+', ' ', regex=True).str.lower()
df_unido['director_name'] = df_unido['director_name'].str.strip().replace(r'\s+', ' ', regex=True).str.lower()

# Ajustes en la limpieza del título
df_unido['title'] = (df_unido['title']
                     .str.replace(r'[^A-Za-z0-9\s.,\'-]', '', regex=True)  # Eliminar símbolos especiales, pero conservar puntos, comas, apóstrofes y guiones
                     .str.replace(r'\s+', ' ', regex=True)  # Normalizar espacios
                     .str.strip()  # Eliminar espacios al inicio y al final
                     .str.lower())  # Convertir a minúsculas

In [476]:
 # Comparar el número de títulos, actores y directores antes y después de la limpieza
num_titles_before = df_unido['title'].nunique()
num_actors_before = df_unido['actor_name'].nunique()
num_directors_before = df_unido['director_name'].nunique()

# Aplicar la limpieza
num_titles_after = df_unido['title'].nunique()
num_actors_after = df_unido['actor_name'].nunique()
num_directors_after = df_unido['director_name'].nunique()

print(f"Antes - Títulos: {num_titles_before}, Actores: {num_actors_before}, Directores: {num_directors_before}")
print(f"Después - Títulos: {num_titles_after}, Actores: {num_actors_after}, Directores: {num_directors_after}")

Antes - Títulos: 1507, Actores: 1321, Directores: 1510
Después - Títulos: 1507, Actores: 1321, Directores: 1510


In [477]:
df_unido.isna().sum()

movie_id                      0
actor_name                    0
director_name                 0
director_movie_id             0
budget                        0
original_language             0
popularity                    0
release_date                  0
revenue                       0
title                         0
vote_average                  0
vote_count                    0
nombre_coleccion              0
generos                       0
production_countries_clean    0
production_companies_clean    0
spoken_languages_clean        0
release_year                  0
dtype: int64

In [478]:
'''# Función para combinar filas por título, priorizando registros con más datos y combinando datos faltantes
def combinar_filas(grupo):
    grupo_completo = grupo.dropna(subset=['budget', 'revenue', 'release_date'])
    
    if not grupo_completo.empty:
        return grupo_completo.iloc[0]
    
    # Combinar datos faltantes con valores disponibles
    fila_combinada = {
        'title': grupo['title'].iloc[0],
        'release_date': grupo['release_date'].dropna().iloc[0] if grupo['release_date'].notna().any() else np.nan,
        'budget': grupo['budget'].dropna().mean() if grupo['budget'].notna().any() else 0,
        'revenue': grupo['revenue'].dropna().mean() if grupo['revenue'].notna().any() else 0,
        'vote_average': grupo['vote_average'].dropna().mean() if grupo['vote_average'].notna().any() else 0,
        'vote_count': grupo['vote_count'].dropna().sum() if grupo['vote_count'].notna().any() else 0,
        'actor_name': ', '.join(grupo['actor_name'].dropna().unique()),
        'director_name': ', '.join(grupo['director_name'].dropna().unique()),
        'generos': grupo['generos'].iloc[0],  # Asegúrate de que las columnas categóricas tengan un valor válido
        'original_language': grupo['original_language'].iloc[0],
        'release_year': grupo['release_year'].iloc[0]
    }
    
    return pd.Series(fila_combinada)

# Aplicar la agrupación por título
df_unido = df_unido.groupby('title').apply(combinar_filas).reset_index(drop=True)

# Verificar títulos únicos después del agrupamiento
print(f"Cantidad de títulos únicos después del agrupamiento: {df_unido['title'].nunique()}")'''

'# Función para combinar filas por título, priorizando registros con más datos y combinando datos faltantes\ndef combinar_filas(grupo):\n    grupo_completo = grupo.dropna(subset=[\'budget\', \'revenue\', \'release_date\'])\n    \n    if not grupo_completo.empty:\n        return grupo_completo.iloc[0]\n    \n    # Combinar datos faltantes con valores disponibles\n    fila_combinada = {\n        \'title\': grupo[\'title\'].iloc[0],\n        \'release_date\': grupo[\'release_date\'].dropna().iloc[0] if grupo[\'release_date\'].notna().any() else np.nan,\n        \'budget\': grupo[\'budget\'].dropna().mean() if grupo[\'budget\'].notna().any() else 0,\n        \'revenue\': grupo[\'revenue\'].dropna().mean() if grupo[\'revenue\'].notna().any() else 0,\n        \'vote_average\': grupo[\'vote_average\'].dropna().mean() if grupo[\'vote_average\'].notna().any() else 0,\n        \'vote_count\': grupo[\'vote_count\'].dropna().sum() if grupo[\'vote_count\'].notna().any() else 0,\n        \'actor_nam

In [479]:
# Eliminar duplicados y conservar el primero
df_unido = df_unido.drop_duplicates(subset=['title'])
print(f"Títulos después de eliminar duplicados: {df_unido.shape[0]}")

# Verificar longitud mínima y máxima de los títulos
df_unido['title_length'] = df_unido['title'].str.len()
longitud_min = df_unido['title_length'].min()
longitud_max = df_unido['title_length'].max()
print(f"Longitud mínima de título: {longitud_min}, Longitud máxima de título: {longitud_max}")
# Filtrar títulos con longitud mínima mayor a 1
df_unido = df_unido[df_unido['title_length'] > 1]
print(f"Títulos después de eliminar los títulos demasiado cortos: {df_unido.shape[0]}")

# Verificar si hay caracteres especiales no deseados en los títulos
caracteres_especiales = df_unido['title'].str.contains(r'[^A-Za-z0-9\s.,\'-]', regex=True).sum()
print(f"Títulos con caracteres especiales no deseados: {caracteres_especiales}")

Títulos después de eliminar duplicados: 1507
Longitud mínima de título: 1, Longitud máxima de título: 86
Títulos después de eliminar los títulos demasiado cortos: 1505
Títulos con caracteres especiales no deseados: 0


In [480]:
# Verificar títulos duplicados
duplicados_title = df_unido['title'].duplicated().sum()
print(f"Cantidad de títulos duplicados: {duplicados_title}")

# Comprobación final de títulos
print(df_unido[['title', 'title_length']].sample(10))

Cantidad de títulos duplicados: 0
                                       title  title_length
18904                       nothing personal            16
14211         wall street money never sleeps            30
12590                      nicholas nickleby            17
5393   where in the world is osama bin laden            37
31831                          sweet nothing            13
20049                       you wont miss me            16
14107                         daddy and them            14
10473                       the missing star            16
6825                            gray matters            12
9322                   the beast of the city            21


In [481]:
print(df_unido[['title']].sample(10))

                           title
20384              anna karenina
16405  je prfre qu'on reste amis
6671          bhaji on the beach
32467          you're telling me
31385               running free
30546                 the garden
39029                   el super
21379   zeitgeist moving forward
45065       reggie watts spatial
37534          hogi pyar ki jeet


In [482]:
# Crear datasets específicos para cada endpoint

# 1. Dataset para cantidad_filmaciones_mes y cantidad_filmaciones_dia
# Dataset para cantidad_filmaciones_mes y cantidad_filmaciones_dia
df_filmaciones = df_unido[['title', 'release_date']].dropna(subset=['release_date'])
df_filmaciones['release_date'] = pd.to_datetime(df_filmaciones['release_date'], errors='coerce')
df_filmaciones = df_filmaciones.dropna(subset=['release_date'])  # Asegurar que las fechas sean válidas
df_filmaciones.to_parquet(r'..\Datasets\filmaciones.parquet')


# Dataset para score_titulo
df_score = df_unido[['title', 'release_date', 'vote_average', 'popularity']].dropna(subset=['vote_average', 'release_date'])
df_score['release_date'] = pd.to_datetime(df_score['release_date'], errors='coerce')
df_score = df_score.dropna(subset=['release_date'])  # Asegurar fechas válidas
df_score.to_parquet(r'..\Datasets\score_titulo.parquet')


# Dataset para votos_titulo
df_votos = df_unido[['title', 'vote_count', 'vote_average', 'release_year']].dropna(subset=['vote_count', 'vote_average'])
df_votos.to_parquet(r'..\Datasets\votos_titulo.parquet')


# Dataset para get_actor
df_actor = df_unido[['actor_name', 'title', 'revenue', 'budget']].dropna(subset=['actor_name'])
df_actor['return'] = df_actor.apply(lambda row: round(row['revenue'] / row['budget'], 5) if row['budget'] > 0 else 0, axis=1)
df_actor = df_actor.dropna(subset=['title', 'budget', 'revenue'])  # Asegurar datos completos
df_actor.to_parquet(r'..\Datasets\actor_dataset.parquet')


# Dataset para get_director
df_director = df_unido[['director_name', 'title', 'release_date', 'revenue', 'budget']].dropna(subset=['director_name'])
df_director['return'] = df_director.apply(lambda row: round(row['revenue'] / row['budget'], 5) if row['budget'] > 0 else 0, axis=1)
df_director = df_director.dropna(subset=['title', 'budget', 'revenue', 'release_date'])  # Asegurar datos completos
df_director.to_parquet(r'..\Datasets\director_dataset.parquet')


# Guardar el dataset completo para propósitos generales
df_unido.to_parquet(r'..\Datasets\dataset_completo.parquet')